# ReAct

![react](react.png)

In [2]:
import datetime

from dotenv import load_dotenv
from langchain_community.chat_models import ChatZhipuAI

load_dotenv()

llm = ChatZhipuAI(model="GLM-4-Plus")

In [3]:
from langchain_community.tools.tavily_search import TavilySearchResults
from langchain_community.tools import GoogleSerperResults
from langchain_community.utilities.tavily_search import TavilySearchAPIWrapper
from langchain_community.utilities import GoogleSerperAPIWrapper

# https://python.langchain.com/docs/integrations/tools/

# TAVILY_API_KEY
# https://api.tavily.com
tavily_search = TavilySearchResults(
    api_wrapper=TavilySearchAPIWrapper(), max_results=3)

# SERPER_API_KEY
# https://serper.dev/api-key
serper_search = GoogleSerperResults(
    api_wrapper=GoogleSerperAPIWrapper(), max_results=3)

# https://duckduckgo.com/ 梯子
# duck_duck_go_search = DuckDuckGoSearchResults(max_results=3, output_format="json")

# https://github.com/searxng/searxng
# https://github.com/searxng/searxng-docker
# searx_search = SearxSearchResults(max_results=3)

query = "明年是中国农历什么年？"

# tavily_result = tavily_search.invoke({"query": query})
# print("tavily_result", tavily_result)
# serper_result = serper_search.invoke({"query": query})
# print("serper_result", serper_result)

In [ ]:
def get_weather(city_name: str):
    """获取天气信息的函数"""
    weather_conditions = {
        '北京': '晴朗',
        '上海': '小雨',
        '广州': '多云',
    }
    return weather_conditions.get(city_name, "未知天气")

print(get_weather('北京'))

In [ ]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langgraph.prebuilt import create_react_agent
import json
from IPython.display import Markdown, display

prompt_template = ChatPromptTemplate.from_messages([
    ("system",
     """你是靠谱的助理。对于给定的目标，制定一个简单的分步计划。
     该计划应涉及单个任务，如果正确执行，将产生正确的答案。不要添加任何多余的步骤。
     最后一步的结果应该是最终答案，如果需要提现时间需明确给出。确保每个步骤都包含所需的所有信息 - 不要跳过步骤。
     必要时调用工具。
     当前时间: {time}"""),
    MessagesPlaceholder("messages")
]).partial(
    time=lambda: datetime.datetime.now().isoformat(),
)

agent_executor = create_react_agent(
    model=llm,
    tools=[get_weather, serper_search],
    state_modifier=prompt_template,
    debug=True
)

response = agent_executor.invoke(
    {"messages": [("user", "明天我要去中国的首都，后天要去中国的金融中心，帮我查询那里的天气。")]}
)

formatted_response = json.dumps(
    response['messages'],
    default=lambda o: o.__dict__ if hasattr(o, '__dict__') else str(o),
    indent=4,
    ensure_ascii=False
)
display(Markdown(f"```json\n{formatted_response}\n```"))